In [ ]:
!apt install neofetch
!neofetch

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  caca-utils chafa fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick imagemagick-6-common imagemagick-6.q16 jp2a libchafa0 libdjvulibre-text
  libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libid3tag0 libidn12 libijs-0.35 libimlib2
  libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra
  libmagickwand-6.q16-6 libnetpbm10 libpci3 libsixel-bin libsixel1 libwmflite-0.2-7 netpbm pci.ids
  pciutils poppler-data toilet toilet-fonts w3m w3m-img
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x imagemagick-doc
  autotrace cups-bsd | lpr | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer
  povray radiance sane-utils texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev
  inkscape poppl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import cv2
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

import collections
import torch
from torch import nn
from torch.utils.data import TensorDataset
import numpy as np
from sklearn.neighbors import NearestNeighbors
import time

%matplotlib inline

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
DATASET_PATH        = "/content/drive/MyDrive/SKRIPSI/dataset/Original"
IMAGE_SIZE          = (224, 224)
CLASS_NAMES         = ['Benign', 'Early', 'Pre', 'Pro']
EPOCHS              = 20
MODEL_PATH          = "/content/drive/MyDrive/SKRIPSI/models/DeepSMOTE/"

In [ ]:
def load_data(separate_class=False):

    images = []
    labels = []

    print("Loading {}".format(DATASET_PATH))

    # Iterate through each folder corresponding to a category
    for i, folder in enumerate(CLASS_NAMES):
        label = i

        # Iterate through each image in our folder
        current_folder_images = []
        current_folder_labels = []
        for file in tqdm(os.listdir(os.path.join(DATASET_PATH, folder))):

            # Get the path name of the image
            img_path = os.path.join(os.path.join(DATASET_PATH, folder), file)

            # Open and resize the img
            image = cv2.imread(img_path, 1)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, IMAGE_SIZE)

            # Append the image and its corresponding label to the output
            current_folder_images.append(image)
            current_folder_labels.append(label)

        current_folder_labels = np.array(current_folder_labels, dtype = 'int32')

        if separate_class:
            images.append(current_folder_images)
            labels.append(current_folder_labels)
        else:
            images.extend(current_folder_images)
            labels.extend(current_folder_labels)

    return (images, labels)

In [ ]:
images, labels = load_data()

Loading /content/drive/MyDrive/SKRIPSI/dataset/Original


100%|██████████| 804/804 [00:36<00:00, 22.07it/s]


In [ ]:
# Define the encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),  # 3x224x224 -> 64x112x112
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # 64x112x112 -> 128x56x56
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),  # 128x56x56 -> 256x28x28
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),  # 256x28x28 -> 512x14x14
            nn.ReLU(),
            nn.Conv2d(512, 1024, kernel_size=4, stride=2, padding=1),  # 512x14x14 -> 1024x7x7
            nn.ReLU()
        )
        self.fc = nn.Linear(1024 * 7 * 7, 2048)  # Flatten to 2048-dimensional latent space

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x

In [ ]:
# Define the decoder
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(2048, 1024 * 7 * 7)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1),  # 1024x7x7 -> 512x14x14
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),  # 512x14x14 -> 256x28x28
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 256x28x28 -> 128x56x56
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 128x56x56 -> 64x112x112
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),  # 64x112x112 -> 3x224x224
            nn.Sigmoid()  # Output in the range [0, 1]
        )

    def forward(self, x):
        x = self.fc(x)
        x = x.view(x.size(0), 1024, 7, 7)  # Reshape to 1024x7x7
        x = self.decoder(x)
        return x

In [ ]:
encoder = Encoder().to(device)
decoder = Decoder().to(device)

#decoder loss function
criterion = nn.MSELoss().to(device)

batch_size = 100

images = torch.tensor(images)
images = images / 255
images = torch.reshape(images, shape=(3256, 3, 224, 224))
labels = torch.tensor(labels)

#torch.Tensor returns float so if want long then use torch.tensor
leukemia_data = TensorDataset(images, labels)
train_loader = torch.utils.data.DataLoader(leukemia_data,
    batch_size=batch_size,shuffle=True)

best_loss = np.inf

enc_optim = torch.optim.Adam(encoder.parameters(), lr = 0.001)
dec_optim = torch.optim.Adam(decoder.parameters(), lr = 0.001)

train_loss_list   = [ 0 for _ in range(EPOCHS) ]
t_mse_loss_list    = [ 0 for _ in range(EPOCHS) ]
t_discr_loss_list  = [ 0 for _ in range(EPOCHS) ]

In [ ]:
for epoch in range(EPOCHS):
    print(f"Epoch : {epoch}")
    train_loss = 0.0
    t_mse_loss = 0.0
    t_discr_loss = 0.0
    # train for one epoch -- set nets to train mode
    encoder.train()
    decoder.train()

    for image,lab in train_loader:

        # zero gradients for each batch
        encoder.zero_grad()
        decoder.zero_grad()
        #print(images)
        image, labs = image.to(device), lab.to(device)
        # print('images ',images.size())
        labsn = labs.detach().cpu().numpy()
        # print('labsn ',labsn.shape, labsn)

        # run images
        z_hat = encoder(image)

        x_hat = decoder(z_hat) #decoder outputs tanh
        # print('xhat ', x_hat.size())
        # print(x_hat)
        mse = criterion(x_hat,image)
        # print('mse ',mse)


        resx = []
        resy = []

        tc = np.random.choice(4, 1)
        # print(tc)
        xbeg = images[labels == tc[0]]
        ybeg = labels[labels == tc[0]]
        # print('xbeg ',xbeg.shape)
        # print('ybeg ',ybeg.shape)
        xlen = len(xbeg)
        nsamp = min(xlen, 100)
        # print('nsamp ', nsamp)
        ind = np.random.choice(list(range(len(xbeg))),nsamp,replace=False)
        # print('ind ', ind)
        xclass = xbeg[ind]
        # print('xclass ', xclass)
        yclass = ybeg[ind]
        # print('yclass ', yclass)

        xclen = len(xclass)
        # print('xclen ',xclen)
        xcminus = np.arange(1,xclen)
        # print('minus ',xcminus.shape,xcminus)

        xcplus = np.append(xcminus,0)
        # print('xcplus ',xcplus)
        # print('xclass ', xclass)
        xcnew = (xclass[[xcplus],:])
        # xcnew = np.squeeze(xcnew)
        xcnew = xcnew.reshape(xcnew.shape[1],xcnew.shape[2],xcnew.shape[3],xcnew.shape[4])
        # print('xcnew ',xcnew.shape)

        xcnew = torch.Tensor(xcnew)
        xcnew = xcnew.to(device)

        # encode xclass to feature space
        xclass = torch.Tensor(xclass)
        xclass = xclass.to(device)
        xclass = encoder(xclass)
        #print('xclass ',xclass.shape)

        xclass = xclass.detach().cpu().numpy()

        xc_enc = (xclass[[xcplus],:])
        xc_enc = np.squeeze(xc_enc)
        #print('xc enc ',xc_enc.shape)

        xc_enc = torch.Tensor(xc_enc)
        xc_enc = xc_enc.to(device)

        ximg = decoder(xc_enc)

        mse2 = criterion(ximg,xcnew)

        comb_loss = mse2 + mse
        comb_loss.backward()

        enc_optim.step()
        dec_optim.step()

        train_loss += comb_loss.item()*image.size(0)
        t_mse_loss += mse.item()*image.size(0)
        t_discr_loss += mse2.item()*image.size(0)


    # print avg training statistics
    train_loss_list[epoch]    = train_loss / len(train_loader)
    t_mse_loss_list[epoch]    = t_mse_loss / len(train_loader)
    t_discr_loss_list[epoch]  = t_discr_loss/len(train_loader)
    print('Train Loss: {:.6f} \tmse loss: {:.6f} \tmse2 loss: {:.6f}'.format(
        train_loss_list[epoch], t_mse_loss_list[epoch], t_discr_loss_list[epoch]))


    if train_loss < best_loss:
        print('Saving..')
        torch.save(encoder.state_dict(), MODEL_PATH + "bst_enc.pth")
        torch.save(decoder.state_dict(), MODEL_PATH + "bst_dec.pth")

        best_loss = train_loss

    print()

Epoch : 0
[1]
[3]
[1]
[0]
[3]
[2]
[3]
[2]
[2]
[3]
[1]
[2]
[3]
[1]
[3]
[0]
[1]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('_mpl-gallery')
# make data
x = np.arange(0, EPOCHS)
# plot
fig, ax = plt.subplots(figsize=(10,10))
# fig, ax = plt.subplots()

# ax.plot(x2, y2 + 2.5, 'x', markeredgewidth=2)
ax.plot(x, train_loss_list, linewidth=2.0)
ax.plot(x, t_discr_loss_list, linewidth=2.0)
ax.plot(x, t_mse_loss_list, linewidth=2.0)
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.legend(['Train','Reconstruction','Penalty'])

ax.set(xlim=(0, 20), xticks=np.arange(1, EPOCHS+1),
       ylim=(0, 20), yticks=np.arange(1, EPOCHS+1))

plt.show()

In [ ]:
!apt install neofetch
!neofetch

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  caca-utils chafa fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick imagemagick-6-common imagemagick-6.q16 jp2a libchafa0 libdjvulibre-text
  libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libid3tag0 libidn12 libijs-0.35 libimlib2
  libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra
  libmagickwand-6.q16-6 libnetpbm10 libpci3 libsixel-bin libsixel1 libwmflite-0.2-7 netpbm pci.ids
  pciutils poppler-data toilet toilet-fonts w3m w3m-img
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x imagemagick-doc
  autotrace cups-bsd | lpr | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer
  povray radiance sane-utils texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev
  inkscape poppl

In [ ]:
!neofetch

            .-/+oossssoo+/-.
        `:+ssssssssssssssssss+:`
      -+ssssssssssssssssssyyssss+-
    .ossssssssssssssssssdMMMNysssso.
   /ssssssssssshdmmNNmmyNMMMMhssssss/
  +ssssssssshmydMMMMMMMNddddyssssssss+
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/
  +sssssssssdmydMMMMMMMMddddyssssssss+
   /ssssssssssshdmNNNNmyNMMMMhssssss/
    .ossssssssssssssssssdMMMNysssso.
      -+sssssssssssssssssyyyssss+-
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.
root@5f52f09be10a 
----------------- 
OS: Ubuntu 22.04.3 LTS x86_64 
Host: Google Compute Engine 
Kernel: 6.1.85+ 
Uptime: 49 mins 
Packages: 1306 (dpkg) 
Shell: bash 5.1.16 
Terminal: jupyter-noteboo 
CPU: Intel Xeon (2) @ 2.199GHz 
Memory: 7921MiB / 